In [10]:
#import pandas as pd
#import datetime as dt
#from pathlib import Path
#import pandas_ta as ta
#from tabulate import tabulate

In [2]:
import pandas as pd
import datetime as dt
from pathlib import Path
import pandas_ta as ta
from tabulate import tabulate

def append_row(df, row):
    return pd.concat([
        df,
        pd.DataFrame([row], columns=row.index)
    ]).reset_index(drop=True)

def getRSI14_and_BB(csvfilename):
    if Path(csvfilename).is_file():
        try:
            df = pd.read_csv(csvfilename)
            if df.empty or 'Close' not in df.columns:
                return 0.00, 0.00, 0.00, 0.00
            else:
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df['rsi14'] = ta.rsi(df['Close'], length=14)
                bb = ta.bbands(df['Close'], length=20)
                if bb is None or df['rsi14'] is None:
                    return 0.00, 0.00, 0.00, 0.00
                df['lowerband'] = bb['BBL_20_2.0']
                df['middleband'] = bb['BBM_20_2.0']
                if pd.isna(df['rsi14'].iloc[-1]) or pd.isna(df['lowerband'].iloc[-1]) or pd.isna(df['middleband'].iloc[-1]):
                    return 0.00, 0.00, 0.00, 0.00
                else:
                    rsival = df['rsi14'].iloc[-1].round(2)
                    ltp = df['Close'].iloc[-1].round(2)
                    lowerband = df['lowerband'].iloc[-1].round(2)
                    middleband = df['middleband'].iloc[-1].round(2)
                return rsival, ltp, lowerband, middleband
        except Exception as e:
            print(f"Error reading {csvfilename}: {e}")
            return 0.00, 0.00, 0.00, 0.00
    else:
        print(f"File does not exist: {csvfilename}")
        return 0.00, 0.00, 0.00, 0.00

def dayweekmonth_datasets(symbol, symbolname):
    if symbol.endswith('.NS'):
        symbol = symbol.replace(".NS", "_NS")

    daylocationstr = f'DATASETS/Daily_Data/{symbol}.csv'
    weeklocationstr = f'DATASETS/Weekly_Data/{symbol}.csv'
    monthlocationstr = f'DATASETS/Monthly_Data/{symbol}.csv'

    cday = dt.datetime.today().strftime('%d/%m/%Y')
    dayrsi14, dltp, daylowerband, daymiddleband = getRSI14_and_BB(daylocationstr)
    weekrsi14, wltp, weeklowerband, weekmiddleband = getRSI14_and_BB(weeklocationstr)
    monthrsi14, mltp, monthlowerband, monthmiddleband = getRSI14_and_BB(monthlocationstr)

    new_row = pd.Series({
        'entrydate': cday,
        'indexcode': symbol,
        'indexname': symbolname,
        'dayrsi14': dayrsi14,
        'weekrsi14': weekrsi14,
        'monthrsi14': monthrsi14,
        'dltp': dltp,
        'daylowerband': daylowerband,
        'daymiddleband': daymiddleband,
        'weeklowerband': weeklowerband,
        'weekmiddleband': weekmiddleband,
        'monthlowerband': monthlowerband,
        'monthmiddleband': monthmiddleband
    })
    return new_row

def generateGFS(scripttype):
    indicesdf = pd.DataFrame(columns=['entrydate', 'indexcode', 'indexname', 'dayrsi14', 'weekrsi14', 'monthrsi14', 'dltp', 'daylowerband', 'daymiddleband', 'weeklowerband', 'weekmiddleband', 'monthlowerband', 'monthmiddleband'])

    fname = f'DATASETS/{scripttype}.csv'
    csvfilename = f'GFS_{scripttype}.csv'
    try:
        with open(fname) as f:
            for line in f:
                if "," not in line:
                    continue
                symbol, symbolname = line.split(",")[0], line.split(",")[1]
                symbol = symbol.replace("\n", "")
                new_row = dayweekmonth_datasets(symbol, symbolname)
                indicesdf = append_row(indicesdf, new_row)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

    indicesdf.to_csv(csvfilename, index=False)
    return indicesdf

df3 = generateGFS('indicesdf')

df4 = df3.loc[
    (df3['monthrsi14'] >= 60.00) &
    (df3['weekrsi14'] >= 60.00) &
    df3['dayrsi14'].between(40, 45) &
    (df3['dltp'] > df3['daylowerband']) &
    (df3['dltp'] < df3['daymiddleband'])
]

df4 = df4.sort_values(by=['dayrsi14'], ascending=True)

if df4.empty:
    print("\033[1mNO INDICES QUALIFIES THE GFS CRITERIA\033[0m")
else:
    print(tabulate(df4, headers='keys', tablefmt='psql', showindex=False))


C:\Users\manoj\AppData\Local\Temp\ipykernel_47452\3866832408.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([


NO INDICES QUALIFIES THE GFS CRITERIA


TESTING


In [5]:
import pandas as pd
from tabulate import tabulate

# Load the indicesdf and indicesstocks files
indicesdf_path = 'indicesdf.csv'
indicesstocks_path = 'indicesstocks.csv'

indicesdf = pd.read_csv(indicesdf_path)
indicesstocks = pd.read_csv(indicesstocks_path, header=None)

# Filter the qualified indices from indicesdf based on criteria
qualified_indices = indicesdf.loc[
    (indicesdf['monthrsi14'] >= 60.00) &
    (indicesdf['weekrsi14'] >= 60.00) &
    indicesdf['dayrsi14'].between(40, 45) &
    (indicesdf['dltp'] > indicesdf['daylowerband']) &
    (indicesdf['dltp'] < indicesdf['daymiddleband'])
]

if qualified_indices.empty:
    print("\033[1mNO INDICES QUALIFIES THE GFS CRITERIA\033[0m")
else:
    # Display the qualified indices in tabular format
    print("Qualified Indices:")
    print(tabulate(qualified_indices, headers='keys', tablefmt='psql', showindex=False))

    # Traverse through indicesstocks to match and process corresponding rows
    for _, qualified_row in qualified_indices.iterrows():
        qualified_index_code = qualified_row['indexcode']

        # Check for a match in indicesstocks (0th column)
        matched_row = indicesstocks[indicesstocks[0] == qualified_index_code]
        if not matched_row.empty:
            # Retrieve comma-separated values from the matching row (1st column)
            matched_values = matched_row.iloc[0, 1]
            if pd.notna(matched_values):
                stocks = matched_values.split(",")

                # Traverse through the stocks
                print(f"\nQualified Index: {qualified_index_code}")
                print("Related Stocks:")
                for stock in stocks:
                    print(f"- {stock.strip()}")
            else:
                print(f"No stocks found for qualified index: {qualified_index_code}")
        else:
            print(f"No match found in indicesstocks for index: {qualified_index_code}")


FileNotFoundError: [Errno 2] No such file or directory: 'indicesdf.csv'